In [75]:
from tqdm import tqdm

from numpy.random import randint

In [3]:
def parse_input(file):
    with open(file) as file_in:
        input_str = file_in.read()

    registers, program = input_str.split('\n\n')
    registers = [rule.split(': ') for rule in registers.splitlines()]
    registers = {id.split(' ')[1]: int(init_value) for id, init_value in registers}
    program = program.split(': ')[1][:-1].split(',')
    program = tuple([int(n) for n in program])

    return program, registers

In [4]:
def compute_combo_operand(operand, registers):
    if 0 <= operand <= 3:
        return operand
    elif operand == 4:
        return registers['A']
    elif operand == 5:
        return registers['B']
    elif operand == 6:
        return registers['C']
    else:
        raise ValueError(f'Operand {operand} is invalid')

In [5]:
def run_program(program, registers):
    ip = 0
    output = []
    while ip < len(program):
        opcode, operand = program[ip], program[ip+1]
        if opcode == 0:
            # adv
            registers['A'] = int(registers['A'] / 2**compute_combo_operand(operand, registers))
        elif opcode == 1:
            # bxl
            registers['B'] = registers['B'] ^ operand
        elif opcode == 2:
            # bst
            registers['B'] = compute_combo_operand(operand, registers) % 8
        elif opcode == 3:
            # jnz
            if registers['A'] != 0:
                ip = operand
                continue
        elif opcode == 4:
            # bxc
            registers['B'] = registers['B'] ^ registers['C']
        elif opcode == 5:
            # out
            out = compute_combo_operand(operand, registers) % 8
            output.append(out)
        elif opcode == 6:
            # bdv
            registers['B'] = int(registers['A'] / 2**compute_combo_operand(operand, registers))
        elif opcode == 7:
            # bdv
            registers['C'] = int(registers['A'] / 2**compute_combo_operand(operand, registers))
        ip += 2

        output_str = ','.join(map(str, output))

    return output_str

In [6]:
def main1(file):
    program, registers = parse_input(file)
    output = run_program(program, registers)
    return output

In [7]:
assert main1('example1.txt') == '4,6,3,5,6,3,5,2,1,0'

In [8]:
main1('input.txt')

'6,0,6,3,0,2,3,1,6'

In [9]:
def main2(file):
    program, registers = parse_input(file)
    program_str = ','.join(map(str, program))
    for i in range(int(1e9)):
        registers_tmp = registers.copy()
        registers_tmp['A'] = i
        if run_program(program, registers_tmp) == program_str:
            return i

In [10]:
assert main2('example2.txt') == 117440

In [92]:
file = 'input.txt'
program, registers = parse_input(file)

inf = int(1e14)
sup = int(1e15)

samples = randint(inf, sup, 10)

for na in samples:
    res = run_program(program, registers={'A': na, "B": 0, 'C': 0})
    print(res)
    if len(res.split(',')) == 16:
        print(na, res)

0,6,7,2,7,6,6,3,1,6,1,6,5,7,5,6,6
3,1,2,1,1,6,6,1,2,7,7,4,6,6,3,0,7
1,6,2,1,4,0,3,0,7,5,6,6,3,6,5,2,5
6,6,4,5,6,5,5,5,3,6,1,5,1,6,2,0
243673655133184 6,6,4,5,6,5,5,5,3,6,1,5,1,6,2,0
1,3,6,1,7,2,6,7,2,7,4,5,2,7,5,1,6
1,2,1,7,5,6,1,3,4,2,0,6,7,2,2,2,7
2,5,2,1,2,5,5,3,7,0,6,6,7,1,6,5,5
0,6,6,0,4,6,6,2,0,5,2,5,1,6,2,6
124878724135661 0,6,6,0,4,6,6,2,0,5,2,5,1,6,2,6
7,3,1,6,4,5,3,0,3,6,7,1,6,1,0,5
101920204744485 7,3,1,6,4,5,3,0,3,6,7,1,6,1,0,5
6,1,6,1,1,2,1,3,0,5,1,1,4,1,6,1,7


In [50]:
def get_out(na):
    return int(i / (2**((i % 8) ^ 3))) % 8

array([3, 6, 5, 5, 3])